### Part 1: Importing the Dataset and  Exploratory Data Analysis (EDA)

Using `pandas` to import the dataset. Importing `pandas` and then reading the file using the `.read_csv` *function* and passing the name of the dataset as a *string*.

Because the rows in the dataset are separated using a `\t`, the type of delimiter is specified in the `.read_csv()` *function*; the default value is `,`. Additionally, the list of column names to use is specified (`"label"` and `"emails"`).

In [1]:
import pandas as pd

emails = pd.read_csv('EmailSpamCollection', sep = '\t', names = ["label", "email"])

Before performing any algorithm on the *dataframe*, it is always good practice to perform exploratory data analysis.

Begining by visualizing the first ten rows of the `df` *dataframe* using the `head()` *function*. By default, *`head()`* displays the first five rows of a *dataframe*.

In [2]:
emails.head(5)

,label,email
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Next, more information is retrieved about the *dataframe* by using the `shape` and `columns` *methods* and the `describe()` *function*.

Here's a brief description of what each of the above *functions* does:
- `shape`: returns a *tuple* representing the dimensionality of the *dataframe*.
- `columns`: returns the column labels of the *dataframe*.
- `describe()`: returns summary statistics of the columns in the *dataframe* provided, such as mean, count, standard deviation, etc.

In [3]:
emails.shape

(5572, 2)

In [4]:
emails.columns

Index(['label', 'email'], dtype='object')

In [5]:
emails.describe()

,label,email
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


### Part 2: Shuffling  and Splitting the Messages

In the second part, emails are shuffled and split them into a training set (2,500 messages), a validation set (1,000 messages), and a test set (all remaining messages).

Shuffling messages is done using the `sample` *function* from the `pandas` *library*.

`frac` denotes the proportion of the *dataframe* to sample, and `random_state` is a random seed that ensures reproducibility of your results. 

Reset the *index* of `emails` to align with the shuffled emails by using the `reset_index` *function* with the appropriate *argument*. 

Here's a link for you where you can find the documentation about the [`reset_index()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html) *function*.

In [6]:
emails = emails.sample(frac = 1, random_state = 0).reset_index(drop = True)

Run the code cell below to visualize the updated *dataframe*.

In [7]:
emails.head()

,label,email
0,ham,"Storming msg: Wen u lift d phne, u say ""HELLO""..."
1,spam,<Forwarded from 448712404000>Please CALL 08712...
2,ham,And also I've sorta blown him off a couple tim...
3,ham,"Sir Goodmorning, Once free call me."
4,ham,All will come alive.better correct any good lo...


Next, emails are split into a training set (the first 2,500 emails in the *dataframe*), a validation set (the next 1,000 emails), and a testing set (the remaining emails). These three sets will be defined as `trainingMsgs`, `valMsgs`, and `testingMsgs`.

In the code cell below, the messages and their correspoding labels are defined. Then messages are split into the required sets according to the instructions.

In [8]:
email = list(emails.email) 
lbls =list(emails.label) 
trainingEmail = email[:2500] 
valEmail = email[2500:3500] 
testingEmail = email[3500:]

The cell below splits the labels into a training set, a validation set, and a test set.

In [9]:
traininglbls = lbls[:2500] 
vallbls = lbls[2500:3500] 
testinglbls = lbls[3500:]

### Part 3: Building a Simple Naïve Bayes Classifier From Scratch

While Python’s SciKit-learn *library* has a [Naïve Bayes classifier](https://scikit-learn.org/stable/modules/naive_bayes.html), it works with continuous probability distributions and assumes numerical features. 

Here is a simple Naïve Bayes classifier built from scratch.

In [10]:
import numpy as np


class NaiveBayesForSpam:
    def train (self, nonSPamMessages, spamMessages):
        self.words = set (' '.join (nonSPamMessages + spamMessages).split())
        self.priors = np.zeros (2)
        self.priors[0] = float (len (nonSPamMessages)) / (len (nonSPamMessages) + len (spamMessages))
        self.priors[1] = 1.0 - self.priors[0]
        self.likelihoods = []
        for i, w in enumerate (self.words):
            prob1 = (1.0 + len ([m for m in nonSPamMessages if w in m])) / len (nonSPamMessages)
            prob2 = (1.0 + len ([m for m in spamMessages if w in m])) / len (spamMessages)
            self.likelihoods.append ([min (prob1, 0.95), min (prob2, 0.95)])
        self.likelihoods = np.array (self.likelihoods).T
        
    def predict (self, message):
        posteriors = np.copy (self.priors)
        for i, w in enumerate (self.words):
            if w in message.lower():  # convert to lower-case
                posteriors *= self.likelihoods[:,i]
            else:                                   
                posteriors *= np.ones (2) - self.likelihoods[:,i]
            posteriors = posteriors / np.linalg.norm (posteriors)  # normalize
        if posteriors[0] > 0.5:
            return ['ham', posteriors[0]]
        return ['spam', posteriors[1]]    

    def score (self, messages, labels):
        confusion = np.zeros(4).reshape (2,2)
        for m, l in zip (messages, labels):
            if self.predict(m)[0] == 'ham' and l == 'ham':
                confusion[0,0] += 1
            elif self.predict(m)[0] == 'ham' and l == 'spam':
                confusion[0,1] += 1
            elif self.predict(m)[0] == 'spam' and l == 'ham':
                confusion[1,0] += 1
            elif self.predict(m)[0] == 'spam' and l == 'spam':
                confusion[1,1] += 1
        return (confusion[0,0] + confusion[1,1]) / float (confusion.sum()), confusion

### Part 4: Explaining the Code Provided in Part 3

Before explaining the code that was provided in Part 3, it is important to have some level of intuition as to what a spam email message might look like. Usually they have words designed to catch your eye and, in some sense, to tempt you to open them. Also, spam emails tend to have words written in all capital letters and use a lot of exclamation marks.

The `train` *function* calculates and stores the prior probabilities and likelihoods based on the training dataset. In Naïve Bayes, this is all the training phase does. 

The `predict` *function* repeatedly applies the Naïve Bayes theorem to every word in the constructed *dictionary* and, based on the posterior probability, it classifies each email as `spam` or `non-spam`. The `score` *function* calls `predict` for multiple emails and compares the outcomes with the supplied `ground truth` labels, thus evaluating the classifier. It also computes and returns a confusion matrix.